In [55]:
signals = pd.DataFrame({'kalman_spread': kfp.spread,
                        'kalman_mean': kfp.mean,
                        'kalman_std': kfp.std,
                        'uob_close': prices['uob_close'].values,
                        'dbs_close': prices['dbs_close'].values})

signals['+1sd'] = signals['kalman_mean'] + signals['kalman_std']
signals['-1sd'] = signals['kalman_mean'] - signals['kalman_std']

In [58]:
def update_cumulative_pos(signals, index):
    signals.loc[index, 'Σuob'] = signals.loc[index-1, 'Σuob'] + signals.loc[index, 'Δuob']
    signals.loc[index, 'Σdbs'] = signals.loc[index-1, 'Σdbs'] + signals.loc[index, 'Δdbs']
    
def keep_cumulative_pos(signals, index):
    signals.loc[index, 'Σuob'] = signals.loc[index-1, 'Σuob']
    signals.loc[index, 'Σdbs'] = signals.loc[index-1, 'Σdbs']

signals['Σuob'] = 0
signals['Δuob'] = 0
signals['Σdbs'] = 0
signals['Δdbs'] = 0

for index, signal in signals.iterrows():
    # we don't trade the first 30 days to give the Kalman Filter time to converge
    if index >= 30:
        # updating variables, feel free to ignore
        exit_short = (signal['kalman_spread'] <= signal['kalman_mean']) and (signals.loc[index-1, 'Σuob'] <= -1)
        exit_long = (signal['kalman_spread'] >= signal['kalman_mean']) and (signals.loc[index-1, 'Σuob'] >= 1)
        enter_short = (signal['kalman_spread'] >= signal['+1sd']) and (signals.loc[index-1, 'Σuob'] == 0)
        enter_long = (signal['kalman_spread'] <= signal['-1sd']) and (signals.loc[index-1, 'Σuob'] == 0)
        
        net_short_uob = -1 - signals.loc[index-1, 'Σuob']
        net_long_uob = 1 - signals.loc[index-1, 'Σuob']
        net_short_dbs = -1 - signals.loc[index-1, 'Σdbs']
        net_long_dbs = 1 - signals.loc[index-1, 'Σdbs']
        exit_all_uob = -signals.loc[index-1, 'Σuob']
        exit_all_dbs = -signals.loc[index-1, 'Σdbs']
        
        # main logic
        # spread has crossed mean and was previous short
        if exit_short:
            signals.loc[index, 'Δuob'] = exit_all_uob # exit all positions
            signals.loc[index, 'Δdbs'] = exit_all_dbs # exit all positions
            update_cumulative_pos(signals, index)

        # spread has crossed mean and was previous long
        if exit_long:
            signals.loc[index, 'Δuob'] = exit_all_uob # exit all positions
            signals.loc[index, 'Δdbs'] = exit_all_dbs # exit all positions
            update_cumulative_pos(signals, index)

        # spread has crossed +1sd, no positions yet
        if enter_short:
            signals.loc[index, 'Δuob'] = net_short_uob # short uob
            signals.loc[index, 'Δdbs'] = net_long_dbs # long dbs
            update_cumulative_pos(signals, index)

        # spread has crossed -1sd, no positions yet
        if enter_long:
            signals.loc[index, 'Δuob'] = net_long_uob # long uob
            signals.loc[index, 'Δdbs'] = net_short_dbs # short dbs
            update_cumulative_pos(signals, index)
            
        if not (exit_short or exit_long or enter_short or enter_long):
            keep_cumulative_pos(signals, index)
            
signals.index = prices.index
signals = signals[~((signals['Δdbs'] == 0) & (signals['Δuob'] == 0))]

In [60]:
signals.head(5)

,kalman_spread,kalman_mean,kalman_std,uob_close,dbs_close,+1sd,-1sd,Σuob,Δuob,Σdbs,Δdbs
Date,,,,,,,,,,,
2000-02-29,-0.372498,0.053214,0.372647,5.88,10.24,0.425861,-0.319433,1,1,-1,-1
2000-03-03,0.081721,0.052034,0.367603,5.78,9.76,0.419637,-0.315569,0,-1,0,1
2000-03-08,-0.346221,0.049371,0.369704,5.29,9.85,0.419075,-0.320333,1,1,-1,-1
2000-03-15,0.229683,0.048916,0.366979,5.51,9.61,0.415895,-0.318064,0,-1,0,1
2000-04-07,0.431542,0.049001,0.376587,6.26,10.96,0.425588,-0.327586,-1,-1,1,1
